### Impor library

In [1]:
from selenium import webdriver
import requests as r
import pandas as pd
from time import time, sleep
from IPython.core.display import clear_output
from warnings import warn
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

### Scraping detik.com

In [2]:
# Webdriver
path = r"C:\Users\Dwi Nissa\AppData\Local\Programs\Python\Python38\Lib\site-packages\selenium\webdriver\firefox\geckodriver.exe"
options = webdriver.FirefoxOptions()
options.set_preference("dom.push.enabled", False)
driver = webdriver.Firefox(executable_path = path, firefox_options=options)

# Set keyword
keyword = "tabrak"; news = []; 
page_begin = 1; page_to_scrap = 603; 
start_time = time(); links = []; titles = []; categories = []

# Dapatkan link berita
for p in range(page_begin, page_to_scrap+1):
    url = "https://www.detik.com/search/searchall?query="+keyword+"&siteid=2&sortby=time&page="+str(p)
    r = driver.get(url); sleep(2)
    boundaries = driver.find_elements(By.XPATH,"/html/body/div[2]/div/div[2]/article"); link_count = 0
    for article in boundaries:
        try:
            link = article.find_element_by_tag_name('a').get_attribute('href')
            if link.split('/')[3]=="berita-jawa-timur":
                links.append(link); link_count +=1
                try:
                    titles.append(article.find_element_by_css_selector("h2[class='title']").text)
                except(NoSuchElementException, IndexError, ValueError): pass
                try:
                    categories.append(article.find_element_by_css_selector(".category").text)
                except(NoSuchElementException, IndexError, ValueError): pass
        except(NoSuchElementException, IndexError, ValueError): pass
    if(len(links)==len(titles)==len(categories)):
        print("get:{}. Total:{}".format(link_count, len(links)))

# Scraping artikel disetiap link
print("Get ready scrap {} of news items..".format(len(links)))
for i in range(len(links)):
    datetime = ""; loc = ""; content = "";
    driver.get(links[i]+'?single=1')
    sleep(2);
    try:
        datetime = driver.find_element_by_css_selector('.detail__date').text
    except(NoSuchElementException, IndexError, ValueError):pass
    try:
        loc = driver.find_element_by_xpath("//div[@class='detail__body-text itp_bodycontent']/strong").text
    except(NoSuchElementException, IndexError, ValueError):pass
    try:
        tag_paragraphs = [paragraph.text for paragraph in driver.find_elements(By.XPATH,"//div[@class='detail__body-text itp_bodycontent']")]
    except(NoSuchElementException, IndexError, ValueError):pass
    content = ' '.join(tag_paragraphs)
    news.append([links[i],datetime,loc,titles[i],content,categories[i]])
print('FINISHED')
data = pd.DataFrame(news,columns = ['Link','Date Time','Location','Title','Content','Category'])
data

Request:131; Frequency:0.07197019524398311 req/sec
Request:132; Frequency:0.0716962936348843 req/sec
Request:133; Frequency:0.0715918589906558 req/sec
Request:134; Frequency:0.0714291920993275 req/sec
Request:135; Frequency:0.07127477207959125 req/sec
Request:136; Frequency:0.07021833772597472 req/sec
Request:137; Frequency:0.06950309867469871 req/sec
Request:138; Frequency:0.06956345530728476 req/sec
-----------------------FINISHED-----------------------
----Scrap Duration: 33.12622579733531 minute(s), in time:1609464885.8147607.----


,Link,Date Time,Location,Title,Content,Category
0,https://news.detik.com/berita-jawa-timur/d-531...,"Kamis, 31 Des 2020 14:51 WIB",Sidoarjo,"Ungkap Kasus Tabrak Lari, Polantas Sidoarjo Da...",Sidoarjo - Kinerja Sat Lantas Polresta Sidoarj...,detikNews
1,https://news.detik.com/berita-jawa-timur/d-531...,"Kamis, 31 Des 2020 09:21 WIB",Ponorogo,Ungkap Kasus Tabrak Lari Terbanyak Berbuah Pen...,Ponorogo - Kasus tabrak lari biasanya membutuh...,detikNews
2,https://news.detik.com/berita-jawa-timur/d-529...,"Rabu, 16 Des 2020 18:11 WIB",Surabaya,Ban Agya Meletus Lalu Tabrak Barrier dan Pikap...,Surabaya - Kecelakaan mobil terjadi di Tol Rom...,detikNews
3,https://news.detik.com/berita-jawa-timur/d-529...,"Rabu, 16 Des 2020 14:35 WIB",Pasuruan,Sopir Truk yang Tewaskan 3 Bikers di Pasuruan ...,Pasuruan - Kecelakaan antara dua motor dan tru...,detikNews
4,https://news.detik.com/berita-jawa-timur/d-529...,"Selasa, 15 Des 2020 17:27 WIB",Pasuruan,Truk Selip Tabrak Dua Motor di Jalur Bromo Pas...,Pasuruan - Kecelakaan maut antara truk dan dua...,detikNews
...,...,...,...,...,...,...
133,https://news.detik.com/berita-jawa-timur/d-485...,"Selasa, 07 Jan 2020 19:54 WIB",Surabaya,"Naik Motor Ugal-ugalan, Pelajar Tewas Setelah ...",Surabaya - Kecelakaan tunggal terjadi di Jalan...,detikNews
134,https://news.detik.com/berita-jawa-timur/d-485...,"Selasa, 07 Jan 2020 19:11 WIB",Surabaya,"Motor Matik Boncengan Tiga Tabrak Trotoar, Sat...",Surabaya - Kecelakaan tunggal terjadi di Jalan...,detikNews
135,https://news.detik.com/berita-jawa-timur/d-484...,"Jumat, 03 Jan 2020 18:22 WIB",Blitar,"Keluarga Tak Mau Urus, Polisi Makamkan Jenazah...",Blitar - Polisi akhirnya memakamkan jenazah ko...,detikNews
136,https://news.detik.com/berita-jawa-timur/d-484...,"Kamis, 02 Jan 2020 19:11 WIB",Pasuruan,SUV Tabrak Barrier Tol Gempol-Pasuruan hingga ...,Pasuruan - Sebuah mobil Toyota Fortuner mengal...,detikNews


In [12]:
import os
FOLDER_DIR = os.path.join(os.getcwd(),'data')
FILENAME = "detik_{}.csv".format(keyword)
FILE_DIR = os.path.join(FOLDER_DIR,FILENAME)
data.to_csv(FILE_DIR)
print("File saved in:{}".format(FILE_DIR))

File saved in:C:\Users\Dwi Nissa\Skripsi\CARI METODE\SCRAPPING\data\detik_tabrak.csv
